In [1]:
import scipy.io
import matplotlib.pyplot as plt
import urllib
import glob
import re
import pandas as pd
import numpy as np

In [2]:
def create_structural_matrix_df(path, pass_end, measure):
    paths_p_tracts = glob.glob(f'{path}/*_{measure}_{pass_end}.mat')
    # initiate atlas and labels
    with open("../data/Schaefer2018_100Parcels_7Networks_order_info.txt", 'r') as f:
        parcels_text = f.read()
    # select only even rows, which contain the labels 
    lines = parcels_text.split('\n')[:-1]
    lines = lines[::2]
    # remove prefix from label: "7Networks_"
    roi_labels = [parcel[10:] for parcel in lines]
    # remove prefix from label: "7Networks_"
    # roi_labels = [roi[10:] for roi in lines]
    
    pattern = re.compile('.*(\d\d\d)_ep2d*') # pattern to find subject id
    df_rows = []
    for path in paths_p_tracts:
        cm = scipy.io.loadmat(path)['CM']
        tril_ind = np.tril_indices(cm.shape[0])
        cm_tril = cm[tril_ind]
        subject = int(re.findall(pattern, path)[0])
        df_row = np.insert(cm_tril, obj=0, values=subject)
        df_rows.append(df_row)
        
    # create label matrix
    multi_ind = list(zip(tril_ind[0], tril_ind[1]))
    label_mat = ['__'.join([roi_labels[i], roi_labels[j]]) for i,j in multi_ind]
    df_columns = np.insert(label_mat, obj=0, values='Subject')
    sc_df = pd.DataFrame(df_rows, columns=df_columns)
    sc_df = sc_df.iloc[sc_df.Subject.argsort(),:].reset_index(drop=True)
    sc_df = sc_df.set_index(sc_df['Subject'].astype(int))
    
    return sc_df

In [5]:
path = '/Volumes/homes/Asaf/processed_data/all_DTIs/all_mat/tracts/CMs/PASS'
sc_df = create_structural_matrix_df(path, 'PASS', 'percentage_of_tracts')

In [9]:
sc_df.mean(axis=0)[1:]

LH_Vis_1__LH_Vis_1                             0.015583
LH_Vis_2__LH_Vis_1                             0.001252
LH_Vis_2__LH_Vis_2                             0.016363
LH_Vis_3__LH_Vis_1                             0.004617
LH_Vis_3__LH_Vis_2                             0.003665
                                                 ...   
RH_Default_pCunPCC_2__RH_Default_PFCdPFCm_1    0.005485
RH_Default_pCunPCC_2__RH_Default_PFCdPFCm_2    0.000989
RH_Default_pCunPCC_2__RH_Default_PFCdPFCm_3         NaN
RH_Default_pCunPCC_2__RH_Default_pCunPCC_1     0.004489
RH_Default_pCunPCC_2__RH_Default_pCunPCC_2     0.019256
Length: 5050, dtype: float64